In [11]:
import numpy as np
import csv
import ast
import random

# TASK 1


In [12]:
def generate_legitimate_error():

    legitimate_error_list = []
    with open('dataset/legitimate_channel_error.csv', newline='') as csvfile:
        
        for line in csvfile:
            legitimate_error_list.append(ast.literal_eval(line.strip()))

    legitimate_error_list_clean = [elem[0] if isinstance(elem, tuple) else elem for elem in legitimate_error_list]

    return legitimate_error_list_clean


def generate_eavesdropper_error():

    eavesdropper_error_list = []
    with open('dataset/eavesdropper_channel_error.csv', newline='') as csvfile:
        
        for line in csvfile:
            eavesdropper_error_list.append(ast.literal_eval(line.strip()))

    eavesdropper_error_list_clean = [elem[0] if isinstance(elem, tuple) else elem for elem in eavesdropper_error_list]

    return eavesdropper_error_list_clean

print(generate_eavesdropper_error())



[[0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 1], [1, 1, 0, 0, 0, 0, 0], [1, 0, 1, 0, 0, 0, 0], [1, 0, 0, 1, 0, 0, 0], [1, 0, 0, 0, 1, 0, 0], [1, 0, 0, 0, 0, 1, 0], [1, 0, 0, 0, 0, 0, 1], [0, 1, 1, 0, 0, 0, 0], [0, 1, 0, 1, 0, 0, 0], [0, 1, 0, 0, 1, 0, 0], [0, 1, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0, 1], [0, 0, 1, 1, 0, 0, 0], [0, 0, 1, 0, 1, 0, 0], [0, 0, 1, 0, 0, 1, 0], [0, 0, 1, 0, 0, 0, 1], [0, 0, 0, 1, 1, 0, 0], [0, 0, 0, 1, 0, 1, 0], [0, 0, 0, 1, 0, 0, 1], [0, 0, 0, 0, 1, 1, 0], [0, 0, 0, 0, 1, 0, 1], [0, 0, 0, 0, 0, 1, 1], [1, 1, 1, 0, 0, 0, 0], [1, 1, 0, 1, 0, 0, 0], [1, 1, 0, 0, 1, 0, 0], [1, 1, 0, 0, 0, 1, 0], [1, 1, 0, 0, 0, 0, 1], [1, 0, 1, 1, 0, 0, 0], [1, 0, 1, 0, 1, 0, 0], [1, 0, 1, 0, 0, 1, 0], [1, 0, 1, 0, 0, 0, 1], [1, 0, 0, 1, 1, 0, 0], [1, 0, 0, 1, 0, 1, 0], [1, 0, 0, 1, 0, 0, 1], [1, 0, 0, 0, 1, 1, 0], [1, 0, 0, 0, 1, 0, 1], [1, 0, 0, 

In [13]:


def random_gen(n):
    if n <= 0:
        raise ValueError("n needs to be > 0")
    return random.randint(0, n - 1)


In [14]:
def binary_sum(x,y):

    result = []
    reminder = 0

    for i in reversed(range(len(x))):
        sum = x[i]+y[i]+reminder
        bit = sum%2
        reminder = sum // 2
        result.insert(0,bit)
    
    if reminder:
        result.insert(0,reminder)
    
    return result


In [15]:
def xor(x,y): #probabilmente si potrebbe usare numpy
    result = []
    for i in range(len(x)):
        result.append(x[i] ^ y[i])
    return result

In [16]:
def wiretap_channel(input):

    legitimate_error_list = generate_legitimate_error()
    eavesdropper_error_list = generate_eavesdropper_error()
    legitimate_error = legitimate_error_list[random_gen(8)]
    eavesdropper_error = eavesdropper_error_list[random_gen(64)]

    return (xor(input,legitimate_error),xor(input,eavesdropper_error))


x = [0,0,0,0,0,0,0]


In [17]:
def run_wiretap_simulation(input, num_runs=10**4):
    results = {}

    for _ in range(num_runs):
        y, z = wiretap_channel(input)

        # Convertiamo le liste x e y in tuple per una gestione più facile nel dizionario
        y_tuple = tuple(y)
        z_tuple = tuple(z)

        if (y_tuple, z_tuple) not in results:
            results[(y_tuple, z_tuple)] = 0
        results[(y_tuple, z_tuple)] += 1

    # Calcoliamo le probabilità
    probabilities = {key: count / num_runs for key, count in results.items()}
    
    return probabilities

# Funzione per dividere le probabilità per x e y
def divide_probabilities(probabilities):
    y_probabilities = {}
    z_probabilities = {}

    # Dividiamo le probabilità
    for (y, z), prob in probabilities.items():
        if y not in y_probabilities:
            y_probabilities[y] = 0
        if z not in z_probabilities:
            z_probabilities[z] = 0
        
        y_probabilities[y] += prob
        z_probabilities[z] += prob

    # Ordinamento delle probabilità
    y_probabilities = dict(sorted(y_probabilities.items()))
    z_probabilities = dict(sorted(z_probabilities.items()))

    return y_probabilities, z_probabilities




In [18]:
probabilities = run_wiretap_simulation(x)
x_probabilities, y_probabilities = divide_probabilities(probabilities)

In [19]:
# Stampa le probabilità ordinate per x e y
print("Probabilità per y:")
for x_val, prob in x_probabilities.items():
    print(f"x = {x_val} : {prob:.6f}")

print("\nProbabilità per z:")
for y_val, prob in y_probabilities.items():
    print(f"y = {y_val} : {prob:.6f}")

Probabilità per y:
x = (0, 0, 0, 0, 0, 0, 0) : 0.120200
x = (0, 0, 0, 0, 0, 0, 1) : 0.117900
x = (0, 0, 0, 0, 0, 1, 0) : 0.128000
x = (0, 0, 0, 0, 1, 0, 0) : 0.119300
x = (0, 0, 0, 1, 0, 0, 0) : 0.131300
x = (0, 0, 1, 0, 0, 0, 0) : 0.126900
x = (0, 1, 0, 0, 0, 0, 0) : 0.128200
x = (1, 0, 0, 0, 0, 0, 0) : 0.128200

Probabilità per z:
y = (0, 0, 0, 0, 0, 0, 0) : 0.018800
y = (0, 0, 0, 0, 0, 0, 1) : 0.015300
y = (0, 0, 0, 0, 0, 1, 0) : 0.012300
y = (0, 0, 0, 0, 0, 1, 1) : 0.016200
y = (0, 0, 0, 0, 1, 0, 0) : 0.015200
y = (0, 0, 0, 0, 1, 0, 1) : 0.014100
y = (0, 0, 0, 0, 1, 1, 0) : 0.015700
y = (0, 0, 0, 0, 1, 1, 1) : 0.016100
y = (0, 0, 0, 1, 0, 0, 0) : 0.016300
y = (0, 0, 0, 1, 0, 0, 1) : 0.017700
y = (0, 0, 0, 1, 0, 1, 0) : 0.014900
y = (0, 0, 0, 1, 0, 1, 1) : 0.015800
y = (0, 0, 0, 1, 1, 0, 0) : 0.017700
y = (0, 0, 0, 1, 1, 0, 1) : 0.014700
y = (0, 0, 0, 1, 1, 1, 0) : 0.016500
y = (0, 0, 1, 0, 0, 0, 0) : 0.016100
y = (0, 0, 1, 0, 0, 0, 1) : 0.016800
y = (0, 0, 1, 0, 0, 1, 0) : 0.016600

# Task 2

In [20]:
H_C = [[0,0,0,0,0,0,0],[0,1,0,0,1,0,1], [0,0,1,0,0,1,1], [0,0,0,1,1,1,1], [0,1,1,0,1,1,0], [0,1,0,1,0,1,0], [0,0,1,1,1,0,0], [0,1,1,1,0,0,1]]


def binary_complement(codeword):

    complement = []
    for i in range (len(codeword)):
        if(codeword[i]==1):
            complement.append(0)
        else:
            complement.append(1)
    return complement

def random_binning_encoder(u):
    mask = [1 for _ in range(7)]
    x = None

    for x in H_C:
        if x[1:4] == u:
            if random.randint(0, 1) == 1:
                x = xor(x, mask)    #calcolca il complemento
            break   
    return x

In [21]:
import itertools


all_messages = [list(vector) for vector in itertools.product([0, 1], repeat=3)]

for m in all_messages:
    print(f"Messaggio: {m}")
    x = random_binning_encoder(m)
    print(f"Codifica: {x}")

Messaggio: [0, 0, 0]
Codifica: [0, 0, 0, 0, 0, 0, 0]
Messaggio: [0, 0, 1]
Codifica: [0, 0, 0, 1, 1, 1, 1]
Messaggio: [0, 1, 0]
Codifica: [1, 1, 0, 1, 1, 0, 0]
Messaggio: [0, 1, 1]
Codifica: [1, 1, 0, 0, 0, 1, 1]
Messaggio: [1, 0, 0]
Codifica: [1, 0, 1, 1, 0, 1, 0]
Messaggio: [1, 0, 1]
Codifica: [1, 0, 1, 0, 1, 0, 1]
Messaggio: [1, 1, 0]
Codifica: [0, 1, 1, 0, 1, 1, 0]
Messaggio: [1, 1, 1]
Codifica: [0, 1, 1, 1, 0, 0, 1]


# Task 3

In [91]:
def hamming_distance(x,y):
    if len(x) != len(y):
        raise ValueError("Le liste devono avere la stessa lunghezza")
    return sum(el_x != el_y for el_x, el_y in zip(x, y))


def random_binning_decoder(y):
    h_c = H_C + [xor([1,1,1,1,1,1,1],x) for x in H_C]
    distances = [hamming_distance(y, x) for x in h_c]
    min_index = distances.index(min(distances))

    x = h_c[min_index]
    if x[0] == 1:
        u = xor(x[1:4], [1,1,1])
    else:
        u = x[1:4]
    return u

In [92]:
all_messages = [list(vector) for vector in itertools.product([0, 1], repeat=3)]

for m in all_messages:
    print(f"Messaggio: {m}")
    x = random_binning_encoder(m)
    print(f"Codifica: {x}")

    u = random_binning_decoder(x)
    print(f"Decodifica: {u}")
    print("-----")

Messaggio: [0, 0, 0]
Codifica: [0, 0, 0, 0, 0, 0, 0]
Decodifica: [0, 0, 0]
-----
Messaggio: [0, 0, 1]
Codifica: [0, 0, 0, 1, 1, 1, 1]
Decodifica: [0, 0, 1]
-----
Messaggio: [0, 1, 0]
Codifica: [0, 0, 1, 0, 0, 1, 1]
Decodifica: [0, 1, 0]
-----
Messaggio: [0, 1, 1]
Codifica: [0, 0, 1, 1, 1, 0, 0]
Decodifica: [0, 1, 1]
-----
Messaggio: [1, 0, 0]
Codifica: [1, 0, 1, 1, 0, 1, 0]
Decodifica: [1, 0, 0]
-----
Messaggio: [1, 0, 1]
Codifica: [0, 1, 0, 1, 0, 1, 0]
Decodifica: [1, 0, 1]
-----
Messaggio: [1, 1, 0]
Codifica: [1, 0, 0, 1, 0, 0, 1]
Decodifica: [1, 1, 0]
-----
Messaggio: [1, 1, 1]
Codifica: [0, 1, 1, 1, 0, 0, 1]
Decodifica: [1, 1, 1]
-----


In [93]:
all_messages = [list(vector) for vector in itertools.product([0, 1], repeat=3)]

for m in all_messages:
    print(f"Messaggio: {m}")
    x = random_binning_encoder(m)
    print(f"Codifica: {x}")
    y, _ = wiretap_channel(x)
    print(f"Canale legittimo: {y}")

    u = random_binning_decoder(y)
    print(f"Decodifica: {u}")
    print("-----")

Messaggio: [0, 0, 0]
Codifica: [0, 0, 0, 0, 0, 0, 0]
Canale legittimo: [0, 0, 0, 1, 0, 0, 0]
Decodifica: [0, 0, 0]
-----
Messaggio: [0, 0, 1]
Codifica: [1, 1, 1, 0, 0, 0, 0]
Canale legittimo: [1, 1, 0, 0, 0, 0, 0]
Decodifica: [0, 0, 1]
-----
Messaggio: [0, 1, 0]
Codifica: [1, 1, 0, 1, 1, 0, 0]
Canale legittimo: [1, 1, 0, 1, 1, 0, 0]
Decodifica: [0, 1, 0]
-----
Messaggio: [0, 1, 1]
Codifica: [1, 1, 0, 0, 0, 1, 1]
Canale legittimo: [1, 1, 0, 0, 0, 0, 1]
Decodifica: [0, 1, 1]
-----
Messaggio: [1, 0, 0]
Codifica: [0, 1, 0, 0, 1, 0, 1]
Canale legittimo: [1, 1, 0, 0, 1, 0, 1]
Decodifica: [1, 0, 0]
-----
Messaggio: [1, 0, 1]
Codifica: [1, 0, 1, 0, 1, 0, 1]
Canale legittimo: [1, 0, 1, 1, 1, 0, 1]
Decodifica: [1, 0, 1]
-----
Messaggio: [1, 1, 0]
Codifica: [1, 0, 0, 1, 0, 0, 1]
Canale legittimo: [1, 0, 0, 0, 0, 0, 1]
Decodifica: [1, 1, 0]
-----
Messaggio: [1, 1, 1]
Codifica: [1, 0, 0, 0, 1, 1, 0]
Canale legittimo: [1, 0, 0, 0, 0, 1, 0]
Decodifica: [1, 1, 1]
-----


#TASK 5

In [94]:
def binary_symmetric_channel(input,error):
    output=[]
    for bit in input:
        if random.random()<=error:
                output.append(bit ^1)
        else:
                output.append(bit)

    return output

def evaluate_BSC(error=0.1,iterations=10**5):
    x = [0,0,0,0,0,0,0]
    #error = 0.1
    count = 0

    for i in range(iterations):
        x_bsc = binary_symmetric_channel(x,error)
        for i in range(7):
            if x_bsc[i]==1:
                count+=1

    print(f"Mean error for {iterations} iterations is: {count/(iterations*len(x))}.\nWith {count} errors!")

#Testing task 5

In [95]:
epsilon = 0.1
delta = 0.3
print("Legitimate channel:")
evaluate_BSC(epsilon,iterations=10**6)
print("Eavesdropper: ")
evaluate_BSC(delta,iterations=10**6)

Legitimate channel:
Mean error for 1000000 iterations is: 0.09993114285714286.
With 699518 errors!
Eavesdropper: 
Mean error for 1000000 iterations is: 0.300112.
With 2100784 errors!


In [96]:
def connected_channels(error,u=[0,0,0,0,0,0,0]):
    x=random_binning_encoder(u)
    y=binary_symmetric_channel(x,error)
    return random_binning_decoder(y)

def connected_channels_simulations(error,u=[0,0,0,0,0,0,0],iterations=10**6):
    
    count=0
    for i in range(iterations):
        if [0,0,0,0,0,0,0] == connected_channels(error):
            count+=1
    return (f"The number of incorrect decoding is: {count} over {iterations} iterations")


In [97]:
connected_channels_simulations(epsilon)

'The number of incorrect decoding is: 0 over 1000000 iterations'

##TASK 6 